
****TOTAL**** 

ADHD : 10
PTSD : 1
GAD : 8
BIPOLAR : 1


In [1]:
# !pip install keras-tuner
# from kerastuner.tuners import RandomSearch
import pandas as pd
import numpy as np 
import random
import math
import matplotlib.pyplot as plt 
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


# TRAINING
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# NEURAL NET
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import optimizers


In [2]:
FEATURES = 15
NO_SAMPLES = 100000
MIN_PER_QUES = 0
MAX_PER_QUES = 4
MAX_SCORE = FEATURES*MAX_PER_QUES
OUTPUT=4
ADHD= 10 
PTSD = 1 
GAD = 8 
BP = 1

# FINAL

In [3]:
df1 = pd.read_csv('../input/results/results_not_normalized.csv' , names=['Q1','Q2','Q3','Q4','Q5','Q6','Q7','Q8','Q9','Q10','Q11','Q12','Q13','Q14','Q15','Q16','Q17' , 'Stress', 'GAD', 'PTSD', 'BP', 'ADHD'])
df1

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q13,Q14,Q15,Q16,Q17,Stress,GAD,PTSD,BP,ADHD
0,0,2,3,2,2,4,4,2,2,2,...,0,0,3,1,1,34,19,1,1,20
1,1,4,0,4,4,1,1,2,4,3,...,2,3,4,1,1,32,16,1,1,24
2,3,4,4,3,4,3,4,1,3,1,...,0,4,3,0,1,35,23,1,0,29
3,4,4,2,4,0,1,1,0,4,3,...,4,2,4,0,0,33,20,0,0,21
4,1,1,1,2,2,4,3,0,3,1,...,0,4,1,1,1,27,16,1,1,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0,3,2,0,3,1,2,2,2,1,...,3,2,0,1,1,19,11,1,1,17
99996,2,1,0,0,2,1,0,0,2,2,...,4,3,3,1,1,28,13,1,1,20
99997,3,1,1,1,4,3,4,2,0,4,...,2,3,1,1,1,32,18,1,1,24
99998,4,3,2,4,2,0,1,3,4,1,...,3,4,0,1,1,35,20,1,1,23


In [4]:
df1.describe()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q13,Q14,Q15,Q16,Q17,Stress,GAD,PTSD,BP,ADHD
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,...,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,2.347710,2.354170,2.348430,2.352790,2.351770,2.349220,2.34592,2.347850,2.345870,2.346880,...,2.347960,2.345570,2.363200,0.624760,0.625590,33.837990,18.789560,0.625590,0.624760,23.507540
std,1.328957,1.327754,1.326885,1.323264,1.323951,1.329347,1.32872,1.328709,1.328805,1.325614,...,1.324787,1.327467,1.326071,0.484187,0.483973,7.118245,5.186652,0.483973,0.484187,6.171545
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,1.000000,0.000000,0.000000,4.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,0.000000,0.000000,28.000000,15.000000,0.000000,0.000000,19.000000
50%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.00000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,1.000000,1.000000,34.000000,19.000000,1.000000,1.000000,23.000000
75%,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.00000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,1.000000,1.000000,40.000000,23.000000,1.000000,1.000000,29.000000
max,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.00000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,1.000000,1.000000,53.000000,32.000000,1.000000,1.000000,39.000000


In [5]:
import pandas as pd


df1 = pd.read_csv('../input/results/results_not_normalized.csv' , names=['Q1','Q2','Q3','Q4','Q5','Q6','Q7','Q8','Q9','Q10','Q11','Q12','Q13','Q14','Q15','Q16','Q17' , 'Stress', 'GAD', 'PTSD', 'BP', 'ADHD'])
X= df1.drop(['Q16', 'Q17', 'Stress', 'GAD', 'PTSD', 'BP', 'ADHD'] , axis=1)
Y= df1[['GAD','ADHD' ]]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)
scalerX = MinMaxScaler()
scalerY = MinMaxScaler()
scaled_Xtrain = scalerX.fit_transform(X_train)
scaled_Xtest = scalerX.transform(X_test)
scaled_ytrain = scalerY.fit_transform(Y_train)
scaled_ytest = scalerY.transform(Y_test)

In [6]:

model = Sequential()
model.add(Dense(80 , input_dim=scaled_Xtrain.shape[1], activation='sigmoid'))
model.add(Dense(scaled_ytrain.shape[1], activation='sigmoid'))
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
history = model.fit(scaled_Xtrain, scaled_ytrain ,validation_data=(scaled_Xtest, scaled_ytest), epochs=5, batch_size=500)

# predict
def predict(input, scalerX=scalerX, model=model):
  scaled_input =  scalerX.transform(input)
  return model.predict(scaled_input)



2022-04-27 19:38:54.759012: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-27 19:38:54.859990: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-27 19:38:54.860822: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-27 19:38:54.861928: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/5
150/150 [==============================] - 2s 3ms/step - loss: 0.7939 - accuracy: 0.5548 - val_loss: 0.7819 - val_accuracy: 0.6150
Epoch 2/5
150/150 [==============================] - 0s 2ms/step - loss: 0.7840 - accuracy: 0.6054 - val_loss: 0.7839 - val_accuracy: 0.6100
Epoch 3/5
150/150 [==============================] - 0s 2ms/step - loss: 0.7866 - accuracy: 0.5860 - val_loss: 0.7841 - val_accuracy: 0.5852
Epoch 4/5
150/150 [==============================] - 0s 2ms/step - loss: 0.7856 - accuracy: 0.5861 - val_loss: 0.7834 - val_accuracy: 0.5945
Epoch 5/5
150/150 [==============================] - 0s 2ms/step - loss: 0.7849 - accuracy: 0.5984 - val_loss: 0.7824 - val_accuracy: 0.6126


In [7]:
data = [[2, 3, 4, 3, 2, 0, 2, 2, 1, 1, 2, 3,2, 2, 3]]
print(predict(data, scalerX, model))

[[1. 1.]]


In [8]:
from kerastuner.tuners import RandomSearch

In [9]:

def tune_model(hp):
  model = Sequential()
  # No. of Hidden layers
  for i in range(hp.Int('num_layers', 2,20)):

    # Dense layers (multiple hidden neurons)
    model.add(Dense(units=hp.Int('units_' + str(i), min_value=10, max_value=512, step=10), 
                    activation='sigmoid' ))
    # model.add(Dense(units=hp.Int('units_' + str(i), min_value=10, max_value=256, step=10), 
    #                 activation='sigmoid' ))
    model.add(Dense(scaled_ytest.shape[1], activation='sigmoid'))
    # Tune for lr
    model.compile(optimizer=optimizers.Adam(hp.Choice('learning_rate', [1e-2,1e-3,1e-4])),
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

tunner = RandomSearch(
    tune_model, 
    objective= 'val_accuracy',
    max_trials =20,
    executions_per_trial = 5,
    directory='tuneClass',
    project_name='IPD Disorders'
)


In [10]:

tunner.search(scaled_Xtrain, scaled_ytrain, epochs=10, validation_data=(scaled_Xtest, scaled_ytest))

Trial 20 Complete [00h 05m 54s]
val_accuracy: 0.8380079984664917

Best val_accuracy So Far: 0.874728012084961
Total elapsed time: 02h 02m 39s


In [11]:
tunner.results_summary()

Results summary
Results in tuneClass/IPD Disorders
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 19
units_0: 410
learning_rate: 0.001
Score: 0.874728012084961
Trial summary
Hyperparameters:
num_layers: 12
units_0: 120
learning_rate: 0.001
Score: 0.8493279933929443
Trial summary
Hyperparameters:
num_layers: 19
units_0: 250
learning_rate: 0.001
Score: 0.8445519924163818
Trial summary
Hyperparameters:
num_layers: 6
units_0: 270
learning_rate: 0.001
Score: 0.8380079984664917
Trial summary
Hyperparameters:
num_layers: 20
units_0: 170
learning_rate: 0.001
Score: 0.8366400003433228
Trial summary
Hyperparameters:
num_layers: 18
units_0: 140
learning_rate: 0.0001
Score: 0.8297119975090027
Trial summary
Hyperparameters:
num_layers: 8
units_0: 280
learning_rate: 0.0001
Score: 0.8267199873924256
Trial summary
Hyperparameters:
num_layers: 16
units_0: 210
learning_rate: 0.001
Score: 0.8171199917793274
Trial summary
Hyperparameters:
